# Scenario 2

Owing to security concerns, a later version of firmware has been rolled out that changes the API. For Reasons, the rollout of the new firmware could not be completed across the board: some routers have been upgraded, some haven't.

### Set up the simulation

In [1]:
# Load the libraries
import os, sys, json
sys.path.insert(0, "../src")
from tutor_router.scenarios import Scenario, NotFoundError

# Set up the scenario
scenario = Scenario.scenario_2()

# Make these look a bit like `requests` calls.
request = scenario.request
get = scenario.get
post = scenario.post

# Get a list of host names, for convenience.
hosts = tuple(scenario.hosts())

In [2]:
hosts

('TutorRouter-1',
 'TutorRouter-2',
 'TutorRouter-3',
 'TutorRouter-4',
 'TutorRouter-5',
 'TutorRouter-6',
 'TutorRouter-7',
 'TutorRouter-8',
 'TutorRouter-9',
 'TutorRouter-10',
 'TutorRouter-11',
 'TutorRouter-12',
 'TutorRouter-13',
 'TutorRouter-14',
 'TutorRouter-15',
 'TutorRouter-16',
 'TutorRouter-17',
 'TutorRouter-18',
 'TutorRouter-19',
 'TutorRouter-20')

## Modifying the Ops script

Let's see what happens when we run the script now, but let's ignore the stats gathering for now...

In [3]:
# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # We need to log in before we can do anything more than health check...
    status_code, response = post(hostname, "login", headers={"username": "admin", "password": "Password123"})
    if status_code != 200:
        print(f"  Failed to log in")
        continue
    auth_token = response

Host TutorRouter-1 is up
  Failed to log in
Host TutorRouter-2 is up
Host TutorRouter-3 is up
Host TutorRouter-4 is not available
Host TutorRouter-5 is up
Host TutorRouter-6 is up
  Failed to log in
Host TutorRouter-7 is up
  Failed to log in
Host TutorRouter-8 is not available
Host TutorRouter-9 is up
Host TutorRouter-10 is up
  Failed to log in
Host TutorRouter-11 is up
Host TutorRouter-12 is up
  Failed to log in
Host TutorRouter-13 is up
  Failed to log in
Host TutorRouter-14 is up
  Failed to log in
Host TutorRouter-15 is not available
Host TutorRouter-16 is not available
Host TutorRouter-17 is not available
Host TutorRouter-18 is up
Host TutorRouter-19 is up
  Failed to log in
Host TutorRouter-20 is up
  Failed to log in


OK well some are still working and others are failing at login; those would be the upgraded routers.

We need some way to fingerprint each router so we know what API calls to make. Luckily the healthcheck can tell us something:

In [4]:
# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        health = get(hostname, "healthcheck")
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up: {health}")

Host TutorRouter-1 is up: (200, '{"host": "TutorRouter-1", "version": 2, "health": "ok"}')
Host TutorRouter-2 is up: (200, '{"host": "TutorRouter-2", "version": 1, "health": "good"}')
Host TutorRouter-3 is up: (200, '{"host": "TutorRouter-3", "version": 1, "health": "good"}')
Host TutorRouter-4 is not available
Host TutorRouter-5 is up: (200, '{"host": "TutorRouter-5", "version": 1, "health": "good"}')
Host TutorRouter-6 is up: (200, '{"host": "TutorRouter-6", "version": 2, "health": "ok"}')
Host TutorRouter-7 is up: (200, '{"host": "TutorRouter-7", "version": 2, "health": "ok"}')
Host TutorRouter-8 is not available
Host TutorRouter-9 is up: (200, '{"host": "TutorRouter-9", "version": 1, "health": "good"}')
Host TutorRouter-10 is up: (200, '{"host": "TutorRouter-10", "version": 2, "health": "ok"}')
Host TutorRouter-11 is up: (200, '{"host": "TutorRouter-11", "version": 1, "health": "good"}')
Host TutorRouter-12 is up: (200, '{"host": "TutorRouter-12", "version": 2, "health": "ok"}')
Ho

Let's use this to use the correct login method accordingly.

In [5]:
# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        status_code, response = get(hostname, "healthcheck")
        health = json.loads(response)
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # Get the firmware version from the healthcheck data
    firmware_version = health["version"]
    
    # We need to log in before we can do anything more than health check...
    if firmware_version == 1:
        status_code, response = post(
            hostname,
            "login",
            headers={"username": "admin", "password": "Password123"}
        )
    elif firmware_version == 2:
        status_code, response = post(
            hostname,
            "authenticate/admin",
            headers={"token": "Password123"}
        )
    if status_code == 200:
        print(f"  Logged in (v{firmware_version})")
    else:
        print(f"  Failed to log in")
        continue
    auth_token = response

Host TutorRouter-1 is up
  Logged in (v2)
Host TutorRouter-2 is up
  Logged in (v1)
Host TutorRouter-3 is up
  Logged in (v1)
Host TutorRouter-4 is not available
Host TutorRouter-5 is up
  Logged in (v1)
Host TutorRouter-6 is up
  Logged in (v2)
Host TutorRouter-7 is up
  Logged in (v2)
Host TutorRouter-8 is not available
Host TutorRouter-9 is up
  Logged in (v1)
Host TutorRouter-10 is up
  Logged in (v2)
Host TutorRouter-11 is up
  Logged in (v1)
Host TutorRouter-12 is up
  Logged in (v2)
Host TutorRouter-13 is up
  Logged in (v2)
Host TutorRouter-14 is up
  Logged in (v2)
Host TutorRouter-15 is not available
Host TutorRouter-16 is not available
Host TutorRouter-17 is not available
Host TutorRouter-18 is up
  Logged in (v1)
Host TutorRouter-19 is up
  Logged in (v2)
Host TutorRouter-20 is up
  Logged in (v2)


Cool, now we can get the interface stats.

In [6]:
# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        status_code, response = get(hostname, "healthcheck")
        health = json.loads(response)
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # Get the firmware version from the healthcheck data
    firmware_version = health["version"]
    
    # We need to log in before we can do anything more than health check...
    if firmware_version == 1:
        status_code, response = post(
            hostname,
            "login",
            headers={"username": "admin", "password": "Password123"}
        )
    elif firmware_version == 2:
        status_code, response = post(
            hostname,
            "authenticate/admin",
            headers={"token": "Password123"}
        )
    if status_code == 200:
        print(f"  Logged in (v{firmware_version})")
    else:
        print(f"  Failed to log in")
        continue
    auth_token = response
    
    # Now grab the interface stats.
    # We have to pass in the auth token because this is a protected operation.
    status_code, response = get(hostname, "interfaces", headers={"Token": auth_token})
    if status_code != 200:
        print(f"  Failed to get interface stats: {response}")
        continue
    print(f"  Interfaces: ({status_code}) {response}")

Host TutorRouter-1 is up
  Logged in (v2)
  Failed to get interface stats: {"message": "Forbidden"}
Host TutorRouter-2 is up
  Logged in (v1)
  Interfaces: (200) {"eth0": {"up": 538953, "down": 111796}, "mgmt": {"up": 142279, "down": 936835}}
Host TutorRouter-3 is up
  Logged in (v1)
  Interfaces: (200) {"eth0": {"up": 426739, "down": 297361}, "mgmt": {"up": 109924, "down": 221617}}
Host TutorRouter-4 is not available
Host TutorRouter-5 is up
  Logged in (v1)
  Interfaces: (200) {"eth0": {"up": 119041, "down": 853904}, "mgmt": {"up": 449801, "down": 489748}}
Host TutorRouter-6 is up
  Logged in (v2)
  Failed to get interface stats: {"message": "Forbidden"}
Host TutorRouter-7 is up
  Logged in (v2)
  Failed to get interface stats: {"message": "Forbidden"}
Host TutorRouter-8 is not available
Host TutorRouter-9 is up
  Logged in (v1)
  Interfaces: (200) {"eth0": {"up": 218115, "down": 926044}, "mgmt": {"up": 818801, "down": 854835}}
Host TutorRouter-10 is up
  Logged in (v2)
  Failed to g

The authentication for calls has changed on V2 routers, so we have to take that into account.

We also know that the response to the query is JSON, no matter which version router we have, so let's do that conversion too.

In [7]:
# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        status_code, response = get(hostname, "healthcheck")
        health = json.loads(response)
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # Get the firmware version from the healthcheck data
    firmware_version = health["version"]
    
    # We need to log in before we can do anything more than health check...
    if firmware_version == 1:
        status_code, response = post(
            hostname,
            "login",
            headers={"username": "admin", "password": "Password123"}
        )
    elif firmware_version == 2:
        status_code, response = post(
            hostname,
            "authenticate/admin",
            headers={"token": "Password123"}
        )
    if status_code == 200:
        print(f"  Logged in (v{firmware_version})")
    else:
        print(f"  Failed to log in")
        continue
    auth_token = response
    
    # Now grab the interface stats.
    # We have to pass in the auth token because this is a protected operation.
    if firmware_version == 1:
        status_code, response = get(
            hostname, "interfaces", headers={"Token": auth_token}
        )
    elif firmware_version == 2:
        status_code, response = get(
            hostname, "interfaces", headers={"Authentication": f"TOKEN {auth_token}"})
    if status_code != 200:
        print(f"  Failed to get interface stats: {response}")
        continue
    
    # Response is always JSON, so let's decode that.
    iface_stats = json.loads(response)
    print(f"  Interfaces: ({status_code}) {iface_stats}")

Host TutorRouter-1 is up
  Logged in (v2)
  Interfaces: (200) {'eth0': {'up': '914 Kbps', 'down': '446 Kbps'}, 'mgmt': {'up': '562 Kbps', 'down': '273 Kbps'}}
Host TutorRouter-2 is up
  Logged in (v1)
  Interfaces: (200) {'eth0': {'up': 538953, 'down': 111796}, 'mgmt': {'up': 142279, 'down': 936835}}
Host TutorRouter-3 is up
  Logged in (v1)
  Interfaces: (200) {'eth0': {'up': 426739, 'down': 297361}, 'mgmt': {'up': 109924, 'down': 221617}}
Host TutorRouter-4 is not available
Host TutorRouter-5 is up
  Logged in (v1)
  Interfaces: (200) {'eth0': {'up': 119041, 'down': 853904}, 'mgmt': {'up': 449801, 'down': 489748}}
Host TutorRouter-6 is up
  Logged in (v2)
  Interfaces: (200) {'eth0': {'up': '216 Kbps', 'down': '333 Kbps'}, 'mgmt': {'up': '401 Kbps', 'down': '458 Kbps'}}
Host TutorRouter-7 is up
  Logged in (v2)
  Interfaces: (200) {'eth0': {'up': '834 Kbps', 'down': '357 Kbps'}, 'mgmt': {'up': '192 Kbps', 'down': '253 Kbps'}}
Host TutorRouter-8 is not available
Host TutorRouter-9 is 

Annoyingly the stats in v2 are strings now and they're counted in **kilo**bits per second! We have to convert them before we can sum them up.

In [8]:
# Iterate over all routers.
for hostname in hosts:
    # Perform a health check on the current router.
    try:
        status_code, response = get(hostname, "healthcheck")
        health = json.loads(response)
    except NotFoundError:
        print(f"Host {hostname} is not available")
        continue
    print(f"Host {hostname} is up")

    # Get the firmware version from the healthcheck data
    firmware_version = health["version"]
    
    # We need to log in before we can do anything more than health check...
    if firmware_version == 1:
        status_code, response = post(
            hostname,
            "login",
            headers={"username": "admin", "password": "Password123"}
        )
    elif firmware_version == 2:
        status_code, response = post(
            hostname,
            "authenticate/admin",
            headers={"token": "Password123"}
        )
    if status_code == 200:
        print(f"  Logged in (v{firmware_version})")
    else:
        print(f"  Failed to log in")
        continue
    auth_token = response
    
    # Now grab the interface stats.
    # We have to pass in the auth token because this is a protected operation.
    if firmware_version == 1:
        status_code, response = get(
            hostname, "interfaces", headers={"Token": auth_token}
        )
    elif firmware_version == 2:
        status_code, response = get(
            hostname, "interfaces", headers={"Authentication": f"TOKEN {auth_token}"})
    if status_code != 200:
        print(f"  Failed to get interface stats: {response}")
        continue
    
    # Response is always JSON, so let's decode that.
    iface_stats = json.loads(response)
    
    # Now let's figure out the throughput
    if firmware_version == 1:
        total_throughput = iface_stats["eth0"]["up"] + iface_stats["eth0"]["down"]
    elif firmware_version == 2:
        total_throughput = (
            int(iface_stats["eth0"]["up"].split()[0]) * 1000 +
            int(iface_stats["eth0"]["down"].split()[0]) * 1000
        )

    # Print the throughput
    print(f"  Throughput: {total_throughput} bits per second")

Host TutorRouter-1 is up
  Logged in (v2)
  Throughput: 1360000 bits per second
Host TutorRouter-2 is up
  Logged in (v1)
  Throughput: 650749 bits per second
Host TutorRouter-3 is up
  Logged in (v1)
  Throughput: 724100 bits per second
Host TutorRouter-4 is not available
Host TutorRouter-5 is up
  Logged in (v1)
  Throughput: 972945 bits per second
Host TutorRouter-6 is up
  Logged in (v2)
  Throughput: 549000 bits per second
Host TutorRouter-7 is up
  Logged in (v2)
  Throughput: 1191000 bits per second
Host TutorRouter-8 is not available
Host TutorRouter-9 is up
  Logged in (v1)
  Throughput: 1144159 bits per second
Host TutorRouter-10 is up
  Logged in (v2)
  Throughput: 854000 bits per second
Host TutorRouter-11 is up
  Logged in (v1)
  Throughput: 1149005 bits per second
Host TutorRouter-12 is up
  Logged in (v2)
  Throughput: 1762000 bits per second
Host TutorRouter-13 is up
  Logged in (v2)
  Throughput: 1118000 bits per second
Host TutorRouter-14 is up
  Logged in (v2)
  Thro

It's a whole lot messier, but we're done now.

Onto how we can rewrite this as [Object Oriented Code](scenario_oop.ipynb)...